<a href="https://www.kaggle.com/code/valyrm/youtube-analysis?scriptVersionId=140322735" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium # Data Visualization
import matplotlib.pyplot as plt # Data Visualization
import seaborn as sns # Data Visualization (Themes)
import geopandas as gpd # Interactive Map with folium
import plotly.express as px

# Input data files are available in the read-only "../input/" directory

#Start by presssing (Shift+Enter)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv


# Youtube Data Analysis

Hey all I'm just starting with python analysis, all feedback is welcomed and appreciated!

**What's my objective?**
* Top 10 youtube channels (location, total count of views, channel creation date, category)
* Top youtube category for maximum number of views
* Create interactive geographical graph


In [2]:
# The defualt encoding can't read the file, therefore change to latin-1
df = pd.read_csv('/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv', encoding = 'latin-1')

#Change the index to rank, this could also be done during .read_csv()
df.set_index('rank', inplace = True)

In [3]:
# Grouping by country to see the sum of subscribers for each country - the top 5 
total_subscribers_country = df.groupby('Country')['subscribers'].sum().sort_values(ascending=False)
total_subscribers_country.head()

Country
United States     7329500000
India             4310600000
Brazil            1221800000
United Kingdom     917400000
Mexico             626900000
Name: subscribers, dtype: int64

Facts: 
* It seems that even though India has some of the most subscribed Youtube accounts the overall number of youtubers and subscribers are in the United States.
* There are a total of 995 Youtube accounts in this dataset.
* There are 18 categories. The top 5 by count are: Entertainment, Music, People & Blogs, Gaming, Comedy

In [4]:
df.head()

,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,video_views_rank,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
rank,,,,,,,,,,,,,,,,,,,,,
1,T-Series,245000000,2.280000e+11,Music,T-Series,20082,India,IN,Music,1.0,...,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
2,YouTube Movies,170000000,0.000000e+00,Film & Animation,youtubemovies,1,United States,US,Games,4055159.0,...,NaN,2006.0,Mar,5.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
3,MrBeast,166000000,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,48.0,...,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
4,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,2.0,...,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
5,SET India,159000000,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,3.0,...,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880


In [5]:
category_count = df.groupby('category')['Youtuber'].count().sort_values(ascending=False).reset_index()
category_count = category_count.rename(columns={'category':'Category', 'Youtuber':'Count of Youtubers'})
category_count.head()

,Category,Count of Youtubers
0,Entertainment,241
1,Music,202
2,People & Blogs,132
3,Gaming,94
4,Comedy,69


In [6]:
# New dataframen to hold the count of Youtubers by Country
selected_columns = ['Youtuber', 'Country', 'Latitude', 'Longitude']

count_Country = df[selected_columns]

count_Country = count_Country.groupby(['Country','Latitude','Longitude'])['Youtuber'].count().reset_index(name='Youtuber_Count').sort_values(by='Youtuber_Count', ascending=False)

fig = px.scatter_geo(count_Country, lat = 'Latitude', lon='Longitude', text='Country', hover_name='Youtuber_Count', color = 'Country', template='ggplot2',
                    color_continuous_scale='deep', title = 'Number of Youtubers per Country (Top 1000)')

fig.update_layout(
    height = 400,
    font=dict(family='Times New Roman', size=13),
    margin=dict(l=0, r=0, b=10, t=50),
 
)

fig.update_geos(
    resolution=110,
    showland=True, landcolor='Beige',
    showocean=True, oceancolor="LightBlue",
)

fig.show()

In [7]:
# Overview of all of the countries in the dataset
df['Country'].unique()

array(['India', 'United States', nan, 'Japan', 'Russia', 'South Korea',
       'United Kingdom', 'Canada', 'Brazil', 'Argentina', 'Chile', 'Cuba',
       'El Salvador', 'Pakistan', 'Philippines', 'Thailand', 'Colombia',
       'Barbados', 'Mexico', 'United Arab Emirates', 'Spain',
       'Saudi Arabia', 'Indonesia', 'Turkey', 'Venezuela', 'Kuwait',
       'Jordan', 'Netherlands', 'Singapore', 'Australia', 'Italy',
       'Germany', 'France', 'Sweden', 'Afghanistan', 'Ukraine', 'Latvia',
       'Switzerland', 'Vietnam', 'Malaysia', 'China', 'Iraq', 'Egypt',
       'Andorra', 'Ecuador', 'Morocco', 'Peru', 'Bangladesh', 'Finland',
       'Samoa'], dtype=object)

# Top 10 Youtubers
In the following section I focus on only the top 10.

The following are the categories by count:<br>
4 - Kids Channels <br>
3 - Music Channels <br>
2 - Personality Channels <br>
1 - Movie Channel <br>


Note: As I went through the list I noticed alot of kids channels - the categories vary between Education, Entertainment, and People & Blogs

In [8]:
# Condense the information I want to display by selecting fewer columns in new dataframe
# Top 10 Youtube Accounts - the year they were created, their category, 

selected_columns = ['Youtuber','subscribers','category','created_year','Country','Latitude', 'Longitude']
top_10_youtubers = df[selected_columns].head(10)
top_10_youtubers

,Youtuber,subscribers,category,created_year,Country,Latitude,Longitude
rank,,,,,,,
1,T-Series,245000000,Music,2006.0,India,20.593684,78.962880
2,YouTube Movies,170000000,Film & Animation,2006.0,United States,37.090240,-95.712891
3,MrBeast,166000000,Entertainment,2012.0,United States,37.090240,-95.712891
4,Cocomelon - Nursery Rhymes,162000000,Education,2006.0,United States,37.090240,-95.712891
5,SET India,159000000,Shows,2006.0,India,20.593684,78.962880
6,Music,119000000,NaN,2013.0,NaN,NaN,NaN
7,ýýý Kids Diana Show,112000000,People & Blogs,2015.0,United States,37.090240,-95.712891
8,PewDiePie,111000000,Gaming,2010.0,Japan,36.204824,138.252924
9,Like Nastya,106000000,People & Blogs,2016.0,Russia,61.524010,105.318756


In [9]:
# Testing out the folium library to create geospatial map - hidden output
map_osm = folium.Map(location = [20.593684, 78.962880], zoom_start = 5)
tooltip="Indian Youtubers"
folium.Marker([20.593684, 78.962880], popup="<i>Total 2: <br> T-Series, SET India</i>", tooltip=tooltip).add_to(map_osm)

map_osm

In [10]:
# Top 10 Youtube accounts grouped by 'Youtube' column, using the longitude and latitude fields we can create a map
aggregated_youtubers = top_10_youtubers.groupby(['Country','Latitude', 'Longitude'])['Youtuber'].apply(', '.join).reset_index()

fig = px.scatter_geo(aggregated_youtubers, lat = 'Latitude', lon='Longitude', text='Country', hover_name='Youtuber', color = 'Country',
                                                  template='ggplot2', title = 'Top 10 Youtubers')

fig.update_layout(
    height = 400,
    font=dict(family='Times New Roman', size=13),
    margin=dict(l=0, r=0, b=10, t=50), 
)

fig.update_geos(
    resolution=110,
    showland=True, landcolor='Beige',
    showocean=True, oceancolor="LightBlue",
)

fig.show()

In [11]:
df.index[df['created_year'] == 1970].tolist()

df = df.drop(102)

In [12]:
# Graph of the year each account was created, any ideas how I could improve this component are appreciated :) 

fig = px.scatter(df, x='created_year', y='Youtuber', color='Youtuber', title='YouTube Account Creation Dates', hover_data=['subscribers',
                                                                                                         'category','channel_type'])
fig.update_xaxes(title_text="Year")

fig.show()

# Conclusion

* Make a kids channel if you want to be successful on Youtube <br>
* PewDiePie moved to Japan